<a href="https://colab.research.google.com/github/Arnaud-Tuynman/gan_sapienza/blob/main/lsgancifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Importation of necessary modules
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
import numpy as np
from numpy import *
from numpy.random import *
#Keras is a deep-learning framework designed for humans, meaning most of the commands are easily understandable. 
#The load_data() function allows to download the data located in the indicated file, in this case cifar10
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import *

from matplotlib import pyplot as plt

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from skimage.transform import resize

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of general variables needed in the code
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

ldim = 100 #This is the dimension that will serve to generate the noise that the generator will use : the latent space
epoch=1500 #This is the number of times the training process will happen

batch=128 #This is the number of images that we will use for one epoch. 
#The discriminator will have to judge a batch coming from the dataset and a batch from the generator.
#The generator will create two batches of images

Image_size=(32,32,3) #Setting the image size of the Cifar10 database : (dimension1, dimension2, black&white(1) or color(3))

Optimizer = Adam(lr=0.0002, beta_1=0.5) #Those values are the best ones for an optimizer according to the DCGAN paper
Loss = 'mse' #This is the mean-squared error. To compare with the binary crossentropy results we write 'binary-crossentropy' instead
Activation = 'linear' #This is the activation function for the LSGAN. If we are using binary crossentropy we change it to 'sigmoid'

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the discriminator model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

discriminator = Sequential()
discriminator.add(Conv2D(64, (5,5), strides=(2, 2), padding='same', input_shape=Image_size))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Conv2D(256, (5,5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.4))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation=Activation))

discriminator.compile(loss=Loss, optimizer=Optimizer, metrics=['accuracy'])

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the generator model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

generator = Sequential()

generator.add(Dense(256*4*4, input_dim=ldim))
generator.add(LeakyReLU(alpha=0.2))
generator.add(Reshape((4, 4, 256)))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
generator.add(LeakyReLU(alpha=0.2))
 
generator.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Definition of the gan model
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#This will be used for generator training
#In order to train the generator, we have to put the discriminator as untrainable
#The discriminator is already compiled so it will not change its training
discriminator.trainable = False 

#The gan is simply composed of both the generator and the discriminator
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss=Loss, optimizer=Optimizer)

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Loading of the Cifar10 database
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#We don't need the validation data for GANs so we leave an underscore meaning it will not be stored
(images, category), (_, _) = load_data()

#Cifar10 dataset has 10 classes of images. For improved results we only choose one of these classes.
#The classes are labeled with numbers from 0 to 9 in the category value.
#Extracting the images from the same category leads to having just one class of images (e.g. for category=7 :horses).
index=np.where(category==7)
dataset=images[index[0]]

#We are shaping the data properly
dataset = dataset.astype('float32')
dataset = (dataset - 127.5) / 127.5

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Training of the models
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
#We are making a loop to train the model for as many epoch as stated
for i in range(epoch):


#In this GAN, the discriminator and the generator are trained at the same pace 
#which is not necessarily the case for all GANs (example the WGAN)


#####Discriminator training#####

	#Choosing real images randomly from the database
	image_index = randint(0, dataset.shape[0], batch)
	real_images = dataset[image_index] #These are the real images
	real_label = ones((batch, 1)) #1 is the label for real images

	#Generating a random noise as input of the generator in order to generate different images
	noise = randn(ldim * batch)
	noise = noise.reshape(batch, ldim)
	fake_images = generator.predict(noise) #This is the fake images
	fake_label = zeros((batch, 1)) #0 is the label for fake images

	#Stacking the real and fake images together for training of the discriminator
	#This function creates 1 array from 2 arrays
	images = vstack((real_images, fake_images))
	labels = vstack((real_label, fake_label))
 
  #The train_on_batch() function runs a gradient update on this batch of data
  #The discriminator gets images as input data and has to guess the labels correctly
  #This function returns the training loss
	discriminator_loss, _ = discriminator.train_on_batch(images, labels)


#####Generator training#####

	#Noise generation as input of the generator training
	noise = randn(ldim * batch * 2)
	noise = noise.reshape(batch*2, ldim)
	#The generated images should aim to be recognized as real by the discriminator, which will be untrainable during the next step
	real_labelg = ones((batch*2, 1))
 
 #We have the noise as input data and the generator will have to create an image that will be classified as real
	generator_loss = gan.train_on_batch(noise, real_labelg)
 
 #The information of the losses of the discriminator and generator is crucial because it gives us information on the training process
 #If the values are too high or equal to 0, something went wrong and the training should be restarted
	print('%d, d=%.3f, g=%.3f' % (i+1, discriminator_loss, generator_loss))


#####Samples verification#####

	#We should regularly do some checkings on the training to see if it is going in the correct direction
	#We are saving a file with some generated images for inspection
	if (i+1) % 100 == 0 :

		_, accuracy_real = discriminator.evaluate(real_images, real_label, verbose=0) #We evaluate the discriminator on the real images
		_, accuracy_fake = discriminator.evaluate(fake_images, fake_label, verbose=0) #We evaluate the discriminator on the fake images

		#Those are the results of the accuracy of the discriminator. 
		#The accuracy on real values should not be equal to 100%, else the learning process becomes saturated.
		print('Accuracy of the discriminator real: %0.f%%, fake: %0.f%%' % (accuracy_real*100, accuracy_fake*100))
	

		#Saving some generated images to look at them
		examples = (fake_images + 1)/2
		n=3 # n*n will be the number of fake images shown
		for k in range(n*n):
			plt.subplot(n, n, 1 + k)
			plt.axis('off')
			plt.imshow(examples[k])
		plt.savefig('Fake_images%03d' % (i+1))
		plt.close()

NameError: ignored

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
Calculating the Inception Score
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#We are shaping the last results in order to evaluate their inception score
result=((examples+1)*127.5).astype('float32')

#InceptionV3() is an image recognition neural network which is used to calculate the inception score
model = InceptionV3()

#The InceptionV3() function accepts only entry in the shape of (299,299,3), so we need to resize all of our images to this shape
data_list = list()
for i in result:
	data = resize(i, (299,299,3))
	data_list.append(data)
 
result=asarray(data_list)

#We need to preprocess the data so that the predict() function from InceptionV3 can work
#It gives us a probability distribution that needs to be transformed in order to get the inception score
#This probability distribution depends on the number of classes perceived by the algorithm

P_yx = model.predict(preprocess_input(result))

#This is the probability that each image is in a class. The sum of this probability for an image is 1
P_y = P_yx.mean(axis=0)

#We are using the Kullback-Leibler divergence with logarithmic probabilities
#This will give us the randomness of the probability we obtained; the less random the higher the inception score will be
#We need the 'e' value in case the probability is 0, which would cause an infinity error
e=1E-16
KL = P_yx * (log(P_yx + e) - log(P_y + e))

#We are making the sum of the values for each image
sum_KL = KL.sum(axis=1)
# Averaging on all the images and adding an exponential to counteract the previous logarithm
#If the average of the sum is close to zero, it means no structure has been detected and the IS will be close to one
IS = exp(mean(sum_KL))

print('Inception score : ', IS)

Inception score :  1.7456135
